In [ ]:
!pip install ../input/fastai017-whl/fastai-2.2.2-py3-none-any.whl
!pip install ../input/fastai017-whl/fastcore-1.3.18-py3-none-any.whl
#!pip install ../input/fastai017-whl/fastai2-0.0.17-py3-none-any.whl

# Imports

In [ ]:
#Load the dependancies
from fastai import *
from fastai.callback.all import *
from fastai.vision.all import *

import seaborn as sns
import numpy as np
import pandas as pd
import os
import cv2
import openslide

from sklearn.model_selection import StratifiedShuffleSplit

sns.set(style="whitegrid")
sns.set_context("paper")

Nach Ausschalten der Internetverbindung, muss auch das vortrainierte Modell manuell geladen werden. Dieses können wir aus dem vorherigen`checkpoint` hereinladen.

In [ ]:
Path('/root/.cache/torch/hub/checkpoints').mkdir(exist_ok=True, parents=True)
!cp '../input/xresnet50-pretrained-weigths/xrn50_940.pth' '/root/.cache/torch/hub/checkpoints/xrn50_940.pth'

In [ ]:
!ls /root/.cache/torch/hub/checkpoints/

In [ ]:
source = Path("../input/prostate-cancer-grade-assessment")
files = os.listdir(source)
print(files)

In [ ]:
train = source/'train_images'
mask = source/'train_label_masks'
train_labels = pd.read_csv(source/'train.csv')

# Verteilung der Datensätze ausgleichen
Wir wollen möglichst die gleiche Anzahl an Trainingsbildern für jede der verschiedenen Tumorklassen (`isup_grade`).

In [ ]:
train_labels.head()

In [ ]:
def plot_count(df, feature, title='', size=2):
    f, ax = plt.subplots(1,1, figsize=(3*size,2*size))
    total = float(len(df))
    sns.countplot(df[feature],order = df[feature].value_counts().index, palette='Set1')
    plt.title(title)
    for p in ax.patches:
        height = p.get_height()
        ax.text(p.get_x()+p.get_width()/2.,
                height + 3,
                '{:1.2f}%'.format(100*height/total),
                ha="center") 
    plt.show()

In [ ]:
plot_count(train_labels, 'isup_grade','ISUP grade - data count and percent', size=3)

Die Verteilung der Trainingsbilder in den einzelnen Klassen ist recht unausgewogen....

isup_0 = train_labels[train_labels.isup_grade == 0]
isup_1 = train_labels[train_labels.isup_grade == 1]
isup_2 = train_labels[train_labels.isup_grade == 2]
isup_3 = train_labels[train_labels.isup_grade == 3]
isup_4 = train_labels[train_labels.isup_grade == 4]
isup_5 = train_labels[train_labels.isup_grade == 5]

print(f'isup_0: {len(isup_0)}, isup_1: {len(isup_1)}, isup_2: {len(isup_2)}, isup_3: {len(isup_3)}, isup_4: {len(isup_4)}, isup_5: {len(isup_5)}')



In [ ]:
isup_0 = train_labels[train_labels.isup_grade == 0]
isup_1 = train_labels[train_labels.isup_grade == 1]
isup_2 = train_labels[train_labels.isup_grade == 2]
isup_3 = train_labels[train_labels.isup_grade == 3]
isup_4 = train_labels[train_labels.isup_grade == 4]
isup_5 = train_labels[train_labels.isup_grade == 5]

print(f'isup_0: {len(isup_0)}, isup_1: {len(isup_1)}, isup_2: {len(isup_2)}, isup_3: {len(isup_3)}, isup_4: {len(isup_4)}, isup_5: {len(isup_5)}')

Mithilfe der Funktion `sample` wählen wir aus jeder Kategorie `1224` zufällige Bilder aus.

In [ ]:
isup_sam0 = isup_0.sample(n=1224)
isup_sam1 = isup_1.sample(n=1224)
isup_sam2 = isup_2.sample(n=1224)
isup_sam3 = isup_3.sample(n=1224)
isup_sam4 = isup_4.sample(n=1224)
isup_sam5 = isup_5.sample(n=1224)

frames = [isup_sam0, isup_sam1, isup_sam2, isup_sam3, isup_sam4, isup_sam5]
balanced_df = pd.concat(frames)
balanced_df

Ein Subsample für schnelleres Training während der Entwicklung erstellen

In [ ]:
#isup_sam0 = isup_0.sample(n=256)
#isup_sam1 = isup_1.sample(n=256)
#isup_sam2 = isup_2.sample(n=256)
#isup_sam3 = isup_3.sample(n=256)
#isup_sam4 = isup_4.sample(n=256)
#isup_sam5 = isup_5.sample(n=256)

#frames = [isup_sam0, isup_sam1, isup_sam2, isup_sam3, isup_sam4, isup_sam5]
#balanced_df = pd.concat(frames)
#balanced_df

In [ ]:
plot_count(balanced_df, 'isup_grade','ISUP grade - data count and percent', size=3)

# Erstellen eines eigenen Trainings- und Testset
Wir wollen aus den vorhandenen Trainingsdaten ein eigenes Testdatenset extrahieren, da das Testdatenset von der Kaggle Competition nicht öffentlich verfügbar ist.

In [ ]:
df_copy = balanced_df.copy()

# 80/20 split or whatever you choose
train_set = df_copy.sample(frac=0.80, random_state=7)
test_set = df_copy.drop(train_set.index)
print(len(train_set), len(test_set))

In [ ]:
#Save train_set to csv for strattification
train_set.to_csv('split.csv', index=False)

## Stratifizierung des Trainingsdatensets

In [ ]:
#df = pd.read_csv('split.csv')
#df.shape[0]

In [ ]:
#sss = StratifiedShuffleSplit(n_splits=6, test_size=0.2, random_state=7)
#strat_df = pd.DataFrame()
#cols=['isup_grade']

#for i, (train_index, test_index) in enumerate(sss.split(df, df.isup_grade)):
#    df_split = df.copy()
#    df_split['fold'] = i
#    df_split.loc[train_index, 'which'] = 'train'
#    df_split.loc[test_index, 'which'] = 'valid'
#    X_train = df_split.loc[train_index]
#    X_valid = df_split.loc[test_index]
#    X_train.loc[:, 'which'] = 'train'
#    X_valid.loc[:, 'which'] = 'valid'
#    
#    mult_dis = X_train.loc[X_train.image_id=='isup_grade']
#    for _ in range(3): X_train = X_train.append(mult_dis)
#    
#    strat_df = strat_df.append(X_train).append(X_valid)
#    print(i, strat_df.shape, [(X_train[c].sum()/len(X_train), X_train[c].sum()) for c in cols])
#strat_df = strat_df.reset_index()

Das nun erstellte stratifizierte Datenset enthält zusätzlich die Information, in welchen `fold` es gehört und ob es in das Trainings- oder Testdatenset gehört.

In [ ]:
#strat_df.head()

## Anzeigen der Bilder aus dem Trainingsset

In [ ]:
def view_image(folder, fn):
    filename = f'{folder}/{fn}.tiff'
    file = openslide.OpenSlide(str(filename))
    t = tensor(file.get_thumbnail(size=(512, 512)))
    pil = PILImage.create(t) 
    return pil

In [ ]:
glee_35 = train_labels[train_labels.gleason_score == '3+5']
glee_35[:5]

In [ ]:
view_image(train, '05819281002c55258bb3086cc55e3b48')

# DataBlock für das Modell erstellen

Für die Verwendung der Bilder im `DataBlock` benötigen wir eine Funktion, die das Bild im `tiff`-Format öffnet und in einen `Tensor` konvertiert.

In [ ]:
def get_i(fn):
    filename = f'{train}/{fn.image_id}.tiff'
    example2 = openslide.OpenSlide(str(filename))
    ee = example2.get_thumbnail(size=(512, 512))
    return tensor(ee)

Nun können wir unseren `DataBlock` definieren.

In [ ]:
blocks = (
          ImageBlock,
          CategoryBlock
          )    
getters = [
           get_i,
           ColReader('isup_grade')
          ]
trends = DataBlock(blocks=blocks,
                  splitter=RandomSplitter(),
                  getters=getters,
                  item_tfms=Resize(512),
                  batch_tfms=aug_transforms(size=224)
                  )

Um unser Modell mit dem `k-fold`-Datenset trainieren zu können, benötigen wir eine eigene Funktion, die für jeden `fold` den `DataLoader` sowie den `Learner`  zurückliefert.

In [ ]:
def train_on_folds(bs, size, base_lr, folds):
    learners = []
    all_val_preds = []
    all_val_labels = []
    all_test_preds = []
    
    for fold in range(folds):
        print(f'Processing fold: {fold}....')
        dls = get_dls(bs=bs, size=size, fold=fold, df=strat_df)
        
        learn = get_learner(dls, arch, loss_func, cbs)
        learn = train_learner(learn, base_lr)
        learn.save(f'model_fold_{fold}')
        learners.append(learn)
        learn.recorder.plot_loss()
        
        test_dl = dls.test_dl(train_df)
        test_preds, _, _ = learn.get_preds(dl=test_dl, with_decoded=True)
        val_preds, val_labels = learn.get_preds()
    
        all_val_preds.append(val_preds)
        all_val_labels.append(val_labels)
        all_test_preds.append(test_preds)
    
    plt.show()
    return learners, all_val_preds, all_val_labels, all_test_preds

In [ ]:
set_seed(7)
def get_dls(bs, size, fold, df):
    df_fold = df.copy()
    df_fold = df_fold.loc[df_fold.fold==fold].reset_index()
    
    trends = DataBlock(blocks=blocks,
                       splitter=IndexSplitter(df_fold.loc[df_fold.which=='valid'].index),
                       getters=getters,
                       item_tfms=Resize(256),
                       batch_tfms=aug_transforms(size=size)
                       )
    dls = trends.dataloaders(df_fold, bs=bs)
    assert (len(dls.train_ds) + len(dls.valid_ds)) == len(df_fold)
    return dls

In [ ]:
def get_learner(dls,arch,loss_func, cbs):
        return cnn_learner(dls,arch,loss_func=loss_func,
                           metrics=metrics, cbs=cbs).to_fp16()

In [ ]:
def train_learner(learn, base_lr):
    learn.fine_tune(10)
    learn.unfreeze()
    learn.fit_one_cycle(10, base_lr)    
    return learn

In [ ]:
#dls = get_dls(bs=128, size=128, fold=2, df=strat_df)

In [ ]:
#dls.show_batch()

In [ ]:
#dls.c

## Basis CNN Learner mit fastai

In [ ]:
bs = 128
dls = trends.dataloaders(df_copy, bs=bs)

In [ ]:
dls.c

In [ ]:
dls.show_batch(max_n=9)

## Cohen`s Kappa

In [ ]:
arch = xresnet54
loss_func = LabelSmoothingCrossEntropy(eps=0.3, reduction='mean')
cbs = [ShowGraphCallback()]
metrics=[accuracy,CohenKappa(weights='quadratic')]
train_df = pd.read_csv('split.csv')
bs = 64
size = 196

In [ ]:
# use this cell for the kfold dataloaders
#dls = get_dls(bs=bs, size=size, fold=1, df=strat_df)

In [ ]:
# use this cell for the non-stratified dataloaders 

In [ ]:
learn = cnn_learner(dls,
                    arch=arch,
                    loss_func=loss_func,
                    metrics=metrics,
                    cbs=cbs)

In [ ]:
learn.lr_find()

In [ ]:
# for the kfold Training
#base_lr = 10e-2
#learners, all_val_preds, all_val_labels, all_test_preds = train_on_folds(bs, size, base_lr, folds=1)

In [ ]:
len(dls.train_ds)

In [ ]:
# for the Training of the Base Learning
learn.fine_tune(15)

In [ ]:
learn.export('./models/prostate1.pth')

### Vollständiges Modell trainineren

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(25)

# Überprüfen anhand des eigenen Testdatensets

In [ ]:
test_set

Beispielbilder aus dem Testdatenset

In [ ]:
tst_dl = dls.test_dl(test_set)
tst_dl.show_batch(max_n=9)

In [ ]:
# If you are using kfold Training
#_, _, pred_classes = learn.get_preds(dl=tst_dl, with_decoded=True)
#pred_classes

In [ ]:
_, _, pred_classes = learn.get_preds(dl=tst_dl, with_decoded=True)

In [ ]:
test_df = test_set.copy()
test_df['isup_grade_pred'] = pred_classes
test_df

## Confusion Matrix

In [ ]:
confusion_matrix = pd.crosstab(test_df['isup_grade'], 
                               test_df['isup_grade_pred'], 
                               rownames=['Actual'], 
                               colnames=['Predicted'])
sns.heatmap(confusion_matrix, annot=True)
plt.show()

In [ ]:
from sklearn.metrics import classification_report 
print(classification_report(test_df['isup_grade'], test_df['isup_grade_pred']))

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
losses,idxs = interp.top_losses()
len(dls.valid_ds)==len(losses)==len(idxs)
interp.plot_confusion_matrix(figsize=(7,7))

# Einreichen der Ergebnisse für den Wettbewerb

In [ ]:
submission_test_path = "../input/prostate-cancer-grade-assessment/test_images/"
sample = '../input/prostate-cancer-grade-assessment/sample_submission.csv'

In [ ]:
!ls ../input/prostate-cancer-grade-assessment

In [ ]:
!touch "../input/prostate-cancer-grade-assessment/test.csv"

In [ ]:
sub_df = pd.read_csv(sample)
test_df = pd.read_csv(source/f'test.csv')
if os.path.exists(submission_test_path):
    #learn.load('prostate.pkl')
    def get_inf(df=test_df):
        filename = f'{submission_test_path}/{df.image_id}.tiff' 
        example2 = openslide.OpenSlide(str(filename))
        ee = example2.get_thumbnail(size=(255, 255))
        return tensor(ee)
    
    blocks = (
          ImageBlock,
          CategoryBlock
          )
    getters = [
           get_inf,
           ColReader('isup_grade')
          ]

    trends = DataBlock(blocks=blocks,
              getters=getters,
              item_tfms=Resize(224)
              )
    
    #dls = trends.dataloaders(test_df, bs=32)
    #learn = cnn_learner(dls, xresnet18)
    
    test_dl = dls.test_dl(test_df)
    _,_,pred_classes = learn.get_preds(dl=test_dl, with_decoded=True)
    
    test_df["isup_grade"] = pred_classes
    sub = test_df[["image_id","isup_grade"]]
    sub.to_csv('submission.csv', index=False)